In [2]:
import tensorflow as tf
tf.VERSION

'1.8.0'

In [3]:
import cv2
cv2.__version__

'4.2.0'

In [4]:
# Compute the 128D vector that describes the face in img identified by
# shape.  In general, if two face descriptor vectors have a Euclidean
# distance between them less than 0.6 then they are from the same
# person, otherwise they are from different people. 
from __future__ import division
from __future__ import print_function

import os
import sys
import math

import cv2
import tqdm
import pickle
import argparse
import numpy as np
import tensorflow as tf
from sklearn import metrics
from scipy import interpolate
from scipy.optimize import brentq

import facenet
from MTCNNWrapper import MTCNNWrapper

class FaceUtil:
    
    def __init__(self, data_path='images', model_dir='model/20180402-114759'):
        self.data_path = data_path
        self.model_dir = model_dir
        self.mtcnn = MTCNNWrapper()
        self.images_placeholder = None
        self.embeddings = None
        self.phase_train_placeholder=None
        self.embedding_size=None
        self.image_size = 160
        self.margin=44
        self.detect_multiple_faces = False



    def convert_to_embedding(self, single=False, img_path=None):
        extracted = []
        with tf.Graph().as_default():
                with tf.Session() as sess:
                    self.sess = sess
                    # Load the model
                    facenet.load_model(self.model_dir)
                    
                    # Get input and output tensors
                    images_placeholder = tf.get_default_graph().get_tensor_by_name("input:0")
                    self.images_placeholder = tf.image.resize_images(images_placeholder,(self.image_size, self.image_size))
                    self.embeddings = tf.get_default_graph().get_tensor_by_name("embeddings:0")
                    self.phase_train_placeholder = tf.get_default_graph().get_tensor_by_name("phase_train:0")
                    self.embedding_size = self.embeddings.get_shape()[1]
                    if not single:
                        for filename in os.listdir(self.data_path):
                            #print ("filename -> ", filename)
                            file_path = os.path.join(self.data_path, filename)
                            img = cv2.imread(file_path, 1)
                            bounding_boxes, points = self.mtcnn.get_result_dict(image=img)
                            faces = self.get_faces(img, bounding_boxes, points, filename)
                            extracted.append(faces)
                        return extracted
                    else:
                        img = cv2.imread(img_path, 1)
                        bounding_boxes, points = self.mtcnn.get_result_dict(image=img)
                        faces = self.get_faces(img, bounding_boxes, points, img_path)
                        return faces
    
    def get_faces(self, img, bounding_boxes, points, filename):
        faces = []
        nrof_faces = bounding_boxes.shape[0]
        #print ('bounding_boxes.shape', bounding_boxes.shape[0])
        #print("No. of faces detected: {}".format(nrof_faces))

        if nrof_faces>0:
            det = bounding_boxes[:,0:4]
            det_arr = []
            img_size = np.asarray(img.shape)[0:2]
            if nrof_faces>1:
                if self.detect_multiple_faces:
                    for i in range(nrof_faces):
                        det_arr.append(np.squeeze(det[i]))
                else:
                    bounding_box_size = (det[:,2]-det[:,0])*(det[:,3]-det[:,1])
                    img_center = img_size / 2
                    offsets = np.vstack([ (det[:,0]+det[:,2])/2-img_center[1], (det[:,1]+det[:,3])/2-img_center[0] ])
                    offset_dist_squared = np.sum(np.power(offsets,2.0),0)
                    index = np.argmax(bounding_box_size-offset_dist_squared*2.0) # some extra weight on the centering
                    det_arr.append(det[index,:])
            else:
                det_arr.append(np.squeeze(det))

            for i, det in enumerate(det_arr):
                det = np.squeeze(det)
                bb = np.zeros(4, dtype=np.int32)
                bb[0] = np.maximum(det[0]-self.margin/2, 0)
                bb[1] = np.maximum(det[1]-self.margin/2, 0)
                bb[2] = np.minimum(det[2]+self.margin/2, img_size[1])
                bb[3] = np.minimum(det[3]+self.margin/2, img_size[0])
                cropped = img[bb[1]:bb[3],bb[0]:bb[2],:]
                resized = cv2.resize(cropped, (self.image_size, self.image_size),interpolation=cv2.INTER_CUBIC)
                prewhitened = facenet.prewhiten(resized)
                faces.append({'name': filename,'rect':[bb[0],bb[1],bb[2],bb[3]],'embedding':self.get_embedding(prewhitened)})

        return faces

    def get_embedding(self, processed_img):
        reshaped = processed_img.reshape(-1, self.image_size, self.image_size, 3)
        feed_dict = {self.images_placeholder:reshaped, self.phase_train_placeholder:False }
        feature_vector = self.sess.run(self.embeddings, feed_dict=feed_dict)
        return feature_vector

    
    def get_eucledian_dist_list(self, emb_list, embedding):
        dist_list = []
        for emb in emb_list:
            dist = np.sqrt(np.sum(np.square(np.subtract(emb[0]['embedding'], embedding[0]['embedding']))))
            #print('distance from {}'.format(emb[0]['name'].split('.')[0]))
            #print('  %1.4f  ' % dist, end='')
            #print("\n")
            dist_list.append([emb[0]['name'], dist])
        return dist_list

    def get_eucledian_dist(self, enc1, enc2):
        dist = np.sqrt(np.sum(np.square(np.subtract(enc1[0]['embedding'], enc2[0]['embedding']))))
        return dist
    def load_data(self):
        if os.path.exists('dbfile'):
            dbfile = open('dbfile', 'rb')
            db = pickle.load(dbfile)
            return db
        return None


if __name__ == '__main__':
    
    face_util = FaceUtil()

    # if pickle file does not exists, generate it
    if not os.path.exists('dbfile'):
        encodings = face_util.convert_to_embedding()

    # read the pickle file
    encodings_list =face_util.load_data()

    embedding = face_util.convert_to_embedding(single=True, img_path='test.JPG')
    encodings_list = face_util.load_data()

    #face_util.get_eucledian_dist_list(encodings_list, embedding)

    

Model directory: model/20180402-114759
Metagraph file: model-20180402-114759.meta
Checkpoint file: model-20180402-114759.ckpt-275
INFO:tensorflow:Restoring parameters from model/20180402-114759\model-20180402-114759.ckpt-275
INFO:tensorflow:Restoring parameters from ./mtcnn_model/all_in_one\mtcnn-3000000
0.4157426357269287


In [5]:
"""Test script that creates database using a pickle file,
  loads the existing database, and has helper methods for 
  verifying and recognising face
"""

import os
import sys
import pickle

from facenet_utils import FaceUtil

database_file_name = 'dbfile'
face_util = FaceUtil()

def store_data(encodings):
    with open('dbfile', 'wb') as f:
        pickle.dump(encodings, f)

def load_data():
    if os.path.exists('dbfile'):
        dbfile = open('dbfile', 'rb')
        db = pickle.load(dbfile)
        return db
    return None

def verify(imgpath1, imgpath2):
    enc1 = face_util.convert_to_embedding(single=True, img_path=imgpath1)
    enc2 = face_util.convert_to_embedding(single=True, img_path=imgpath2)
    dist = face_util.get_eucledian_dist(enc1, enc2)
    print ('distance is %1.4f' % dist)
    
def who_it_is(img_path):
    # create database pickle file, if not exists
    if not os.path.exists('dbfile'):
        encodings = face_util.convert_to_embedding()
        store_data(encodings)

    # read the pickle file
    enc_list = load_data()

    test_img_enc = face_util.convert_to_embedding(single=True, img_path=img_path)
    dist_list = face_util.get_eucledian_dist_list(enc_list, test_img_enc)

    #print (dist_list)
    flag = 0
    for item in dist_list:
        if item[1] <= 0.8:
            name = item[0].split('.')[0]
            print ("Welcome {}!!".format(name))
            flag = 1
            break
    if flag == 0:
        print ('No Record found in Database!!')
        print ('Intruder')


if __name__ == "__main__":

    #verify('./images/Roark.jpg', 'test.jpg')
    who_it_is('test.jpg')




Model directory: model/20180402-114759
Metagraph file: model-20180402-114759.meta
Checkpoint file: model-20180402-114759.ckpt-275
INFO:tensorflow:Restoring parameters from model/20180402-114759\model-20180402-114759.ckpt-275
INFO:tensorflow:Restoring parameters from ./mtcnn_model/all_in_one\mtcnn-3000000
0.365886926651001
Welcome Roark!!


In [6]:
import pandas as pd

In [7]:
dbfile = open('dbfile', 'rb')
db = pickle.load(dbfile)

In [8]:
db

[[{'name': 'Amit.jpg',
   'rect': [327, 284, 705, 744],
   'embedding': array([[ 2.97342986e-02,  3.68355475e-02, -4.79619280e-02,
            1.68019682e-02, -2.54687611e-02, -2.13633589e-02,
           -2.89152898e-02,  5.32064438e-02, -1.15882233e-03,
            1.01850154e-02,  2.26945486e-02, -3.80454324e-02,
           -4.20315303e-02, -2.32798816e-03,  5.72686717e-02,
            4.32067178e-02, -8.29712022e-03,  3.55003774e-02,
            2.46847011e-02,  4.23574708e-02,  4.14844714e-02,
           -2.01561116e-03,  4.79751192e-02,  3.50813046e-02,
            4.78250645e-02,  3.78059857e-02,  1.24089345e-02,
           -1.62666496e-02,  5.46918362e-02,  1.64587740e-02,
           -8.59809741e-02,  3.11427936e-03, -7.16934949e-02,
            4.95643094e-02,  5.85476682e-02,  6.57699481e-02,
            3.22590321e-02,  1.18645551e-02,  4.44006585e-02,
            5.01936488e-02, -9.79070272e-03,  3.17940004e-02,
           -4.41227406e-02,  2.78162621e-02, -2.58021150e-02,
 

In [9]:
df = pd.DataFrame([])
for i in db:
    #print(i)
    df = df.append(i)
    

In [10]:
df

,name,rect,embedding
0,Amit.jpg,"[327, 284, 705, 744]","[[0.029734299, 0.036835548, -0.047961928, 0.01..."
0,Annindya.jpg,"[128, 173, 524, 697]","[[0.04921504, 0.052303016, 0.00577977, 0.06251..."
0,Bale.jpg,"[13, 28, 133, 174]","[[0.0388889, -0.018338162, -0.0381498, -0.0663..."
0,Bunty.jpg,"[233, 425, 526, 791]","[[0.06627123, -0.016412983, 0.0038776943, 0.02..."
0,dhiraj_r.jpg,"[87, 71, 363, 400]","[[0.05751693, -0.0067409067, -0.003380234, 0.0..."
0,Drogo.jpg,"[93, 45, 350, 380]","[[0.006210779, -0.100390956, -0.07914555, -0.0..."
0,Edison.jpg,"[237, 252, 604, 704]","[[0.017451204, 0.03659577, -0.06442854, 0.0344..."
0,Heath.jpg,"[0, 20, 88, 136]","[[0.0064863693, -0.009102217, -0.09046058, 0.0..."
0,Jamie.jpg,"[18, 2, 112, 105]","[[-0.014847419, -0.04382121, -0.014748255, -0...."
0,Jon.jpg,"[27, 31, 135, 155]","[[-0.016033491, -0.00017553155, -0.037806734, ..."
